In [2]:
#Prüft in welcher Phase sich der Spieler befindet
def get_player_phase():
    #ToDo: Implementieren
    return

In [4]:
#Berechnet die nächsten Möglichen klickbaren Positionen, falls der Spieler in Phase 1 (Setzphase) ist
def states_phase_one():
    #ToDo: Implementieren
    return

In [6]:
#Berechnet die nächsten Möglichen klickbaren Positionen, falls der Spieler in Phase 2 (Zugphase) ist
def states_phase_two():
    #ToDo: Implementieren
    return

In [7]:
#Berechnet die nächsten Möglichen klickbaren Positionen, falls der Spieler in Phase 3 (Springphase) ist
def states_phase_three():
    #ToDo: Implementieren
    return

In [8]:
#Berechnet in welcher Phase sich der Spieler befindet und dementsprechen, wo er als nächstes klicken kann
# Kombiniert also die Funktionen oben
def next_states():
    #ToDo: Implementieren
    return

In [10]:
def find_muehlen():
    #ToDo: Implementieren
    return